# Experiment:

Explore the dataset and build a model that can accept the data

In [1]:
import pandas as pd
import numpy as np
from sklearn.model_selection import train_test_split
from utils import *
import argparse
import os

In [2]:
import torch
import torch.nn as nn
import torch.nn.init as init
from torch.autograd import Variable
import torch.optim as optim
from torch.utils.data import DataLoader
import torchvision.datasets as datasets
import torchvision.transforms as transforms
from torchvision.utils import save_image
#from torchnet.meter import AverageValueMeter
import torch.backends.cudnn as cudnn


In [3]:
from model import *

In [4]:
parser = {
    'data_dir': './selfdrivingcar-data/lap1/',
    'nb_epoch': 50,
    'test_size': 64,
    'learning_rate': 0.0001,
    'keep_prob': 136,
    'samples_per_epoch': 64,
    'batch_size': 100,
    'save_best_only': 64,
    'cuda': True,
    'seed': 7
}
args = argparse.Namespace(**parser)
args.cuda = args.cuda and torch.cuda.is_available()

torch.manual_seed(args.seed)
if args.cuda:
    torch.cuda.manual_seed(args.seed)

In [5]:
def load_data(args):
    """
    Load training data and split it into training and validation set
    """
    #reads CSV file into a single dataframe variable
    data_df = pd.read_csv(os.path.join(os.getcwd(), args.data_dir, 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

    #yay dataframes, we can select rows and columns by their names
    #we'll store the camera images as our input data
    X = data_df[['center', 'left', 'right']].values
    #and our steering commands as our output data
    y = data_df['steering'].values

    #now we can split the data into a training (80), testing(20), and validation set
    #thanks scikit learn
    X_train, X_valid, y_train, y_valid = train_test_split(X, y, test_size=args.test_size, random_state=0)

    return X_train, X_valid, y_train, y_valid

In [6]:
X_train, X_valid, y_train, y_valid = load_data(args)

In [7]:
transformations = transforms.Compose([transforms.ToTensor(),
                                     transforms.Normalize((127.5, 127.5, 127.5), (127.5, 127.5, 127.5))
                                     ])

In [8]:
train_set = CarDataset(X_train, y_train, args.data_dir, True, transformations)

In [9]:
train_loader = DataLoader(train_set, batch_size=args.batch_size, shuffle=True, num_workers=4)

test_image = 0
for data, steer in train_loader:
    print(data[0])
    print(steer[0])
    test_image = data[0]
    break

# Build Model

 ### NVIDIA model used
    Image normalization to avoid saturation and make gradients work better.
    Convolution: 5x5, filter: 24, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 36, strides: 2x2, activation: ELU
    Convolution: 5x5, filter: 48, strides: 2x2, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Convolution: 3x3, filter: 64, strides: 1x1, activation: ELU
    Drop out (0.5)
    Fully connected: neurons: 100, activation: ELU
    Fully connected: neurons: 50, activation: ELU
    Fully connected: neurons: 10, activation: ELU
    Fully connected: neurons: 1 (output)
    
    the convolution layers are meant to handle feature engineering
    the fully connected layer for predicting the steering angle.
    dropout avoids overfitting
    ELU(Exponential linear unit) function takes care of the Vanishing gradient problem. 

# Define Train

In [10]:
# Training
def train(epoch, net, dataloader, optimizer, criterion, use_cuda):
    print('\nEpoch: %d' % epoch)
    net.train()
    train_loss = 0
    correct = 0
    total = 0
    for batch_idx, (inputs, targets) in enumerate(dataloader):
        optimizer.zero_grad()
        inputs, targets = Variable(inputs), Variable(targets.float())
        if use_cuda:
            inputs, targets = inputs.cuda(), targets.cuda()
        outputs = net(inputs)
        loss = criterion(outputs, targets)
        loss.backward()
        optimizer.step()

        train_loss += loss.data[0]
        print('Loss: %.3f '
            % (train_loss/(batch_idx+1)))

In [11]:
net = CarModel()
optimizer = optim.Adam(net.parameters(), lr=args.learning_rate, weight_decay=5e-4)

if args.cuda:
    net.cuda()
    net = torch.nn.DataParallel(net, device_ids=range(torch.cuda.device_count()))
    cudnn.benchmark = True

criterion = nn.MSELoss()

In [12]:
for epoch in range(0,50):
    #optimizer = lr_scheduler(optimizer, epoch, lr_decay_epoch=args.lr_decay_epoch)	
    train(epoch, net, train_loader, optimizer, criterion, args.cuda)
    #test(epoch, net, criterion)


Epoch: 0
Loss: 0.043 
Loss: 0.043 
Loss: 0.042 
Loss: 0.039 
Loss: 0.039 
Loss: 0.039 
Loss: 0.041 
Loss: 0.042 
Loss: 0.044 
Loss: 0.044 
Loss: 0.045 
Loss: 0.048 
Loss: 0.049 
Loss: 0.048 

Epoch: 1
Loss: 0.048 
Loss: 0.050 
Loss: 0.048 
Loss: 0.047 
Loss: 0.049 
Loss: 0.047 
Loss: 0.047 
Loss: 0.045 
Loss: 0.046 
Loss: 0.047 
Loss: 0.047 
Loss: 0.050 
Loss: 0.050 
Loss: 0.050 

Epoch: 2
Loss: 0.026 
Loss: 0.034 
Loss: 0.040 
Loss: 0.043 
Loss: 0.042 
Loss: 0.047 
Loss: 0.046 
Loss: 0.046 
Loss: 0.048 
Loss: 0.048 
Loss: 0.049 
Loss: 0.053 
Loss: 0.053 
Loss: 0.052 

Epoch: 3
Loss: 0.034 
Loss: 0.034 
Loss: 0.041 
Loss: 0.042 
Loss: 0.047 
Loss: 0.047 
Loss: 0.047 
Loss: 0.045 
Loss: 0.046 
Loss: 0.048 
Loss: 0.048 
Loss: 0.048 
Loss: 0.048 
Loss: 0.048 

Epoch: 4
Loss: 0.057 
Loss: 0.038 
Loss: 0.047 
Loss: 0.045 
Loss: 0.045 
Loss: 0.046 
Loss: 0.047 
Loss: 0.045 
Loss: 0.047 
Loss: 0.048 
Loss: 0.050 
Loss: 0.051 
Loss: 0.052 
Loss: 0.050 

Epoch: 5
Loss: 0.051 
Loss: 0.051 
Loss

Loss: 0.048 
Loss: 0.048 
Loss: 0.049 
Loss: 0.050 
Loss: 0.050 
Loss: 0.051 

Epoch: 43
Loss: 0.050 
Loss: 0.042 
Loss: 0.045 
Loss: 0.045 
Loss: 0.048 
Loss: 0.050 
Loss: 0.047 
Loss: 0.047 
Loss: 0.048 
Loss: 0.047 
Loss: 0.048 
Loss: 0.048 
Loss: 0.048 
Loss: 0.051 

Epoch: 44
Loss: 0.044 
Loss: 0.043 
Loss: 0.039 
Loss: 0.044 
Loss: 0.042 
Loss: 0.043 
Loss: 0.049 
Loss: 0.050 
Loss: 0.050 
Loss: 0.050 
Loss: 0.049 
Loss: 0.051 
Loss: 0.050 
Loss: 0.050 

Epoch: 45
Loss: 0.057 
Loss: 0.052 
Loss: 0.048 
Loss: 0.049 
Loss: 0.049 
Loss: 0.050 
Loss: 0.050 
Loss: 0.050 
Loss: 0.051 
Loss: 0.050 
Loss: 0.050 
Loss: 0.049 
Loss: 0.050 
Loss: 0.049 

Epoch: 46
Loss: 0.048 
Loss: 0.052 
Loss: 0.047 
Loss: 0.048 
Loss: 0.046 
Loss: 0.044 
Loss: 0.045 
Loss: 0.043 
Loss: 0.043 
Loss: 0.048 
Loss: 0.049 
Loss: 0.049 
Loss: 0.048 
Loss: 0.048 

Epoch: 47
Loss: 0.046 
Loss: 0.057 
Loss: 0.056 
Loss: 0.051 
Loss: 0.050 
Loss: 0.051 
Loss: 0.052 
Loss: 0.050 
Loss: 0.051 
Loss: 0.051 
Loss: 0.0

## Save Model

In [13]:
state = {
        'net': net.module if args.cuda else net,
        }

In [14]:
torch.save(state, './model.h5')

In [15]:
data_df = pd.read_csv(os.path.join(os.getcwd(), args.data_dir, 'driving_log.csv'), names=['center', 'left', 'right', 'steering', 'throttle', 'reverse', 'speed'])

In [16]:
data_df

,center,left,right,steering,throttle,reverse,speed
0,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000078
1,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000079
2,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000079
3,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000078
4,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000080
5,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000079
6,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000079
7,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000081
8,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000079
9,/mnt/data/github_dir/car-behavioral-cloning-wi...,/mnt/data/github_dir/car-behavioral-cloning-w...,/mnt/data/github_dir/car-behavioral-cloning-w...,0.000000,0.000000,0,0.000078
